# Agentic Patterns Exercise

In [ ]:
# Imports
from langgraph.graph import START, END, StateGraph
from langchain_core.messages import HumanMessage

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal, TypedDict
from pydantic import BaseModel, Field

import os

print("All imports successful")

## Adaptive Reflection with Quality Metrics

In [ ]:
load_dotenv()
api_key = os.getenv("paid_api")

if not api_key:
    raise ValueError("API_Key not found. Please set it in your .env file")
print("API key loaded")

In [ ]:
## Initialize LLM
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature=0.5,
    api_key = api_key
)
print(f"LLM initialized: {llm.model_name}")

### Reflection Pattern

In [ ]:
class QualityScore(BaseModel):
    """Quality scoring model."""
    clarity: int = Field(ge=1, le=5)
    completeness: int = Field(ge=1, le=5)
    accuracy: int = Field(ge=1, le=5)

    def is_high_quality(self) -> bool:
        """Check if all scores are >= 4."""
        return all(score >= 4 for score in [self.clarity, self.completeness, self.accuracy])

class MetricReflectionState(TypedDict):
    """State for metric-based reflection."""
    task: str
    draft: str
    iterations: int
    scores: list[QualityScore]
    critique: str
    iterations: int
    final_output: str

MAX_METRIC_ITERATIONS = 3
QUALITY_THRESHOLD = 4



In [ ]:
def metric_generator(state: MetricReflectionState) -> dict:
    """Generate or refine based on critique."""
    if state["iterations"] == 0:
        prompt = f"Create a response for: {state['task']}"
        print("\nGenerating initial draft...")
    else:
        prompt = f"""Improve this draft:

Task: {state['task']}
Draft: {state['draft']}
Critique: {state['critique']}

Create improved version."""
        print(f"\nRefining (iteration {state['iterations']})...")
    
    response = llm.invoke([HumanMessage(content=prompt)])
    print("✓ Draft created\n")
    return {
        "draft": response.content
    } #Include iteration


def metric_critic(state: MetricReflectionState) -> dict:
    """Score draft on quality metrics."""
    llm_with_structure = llm.with_structured_output(QualityScore)
    
    prompt = f"""Evaluate this response:

Task: {state['task']}
Response: {state['draft']}

Score on:
- Clarity (1-5): How clear and understandable?
- Completeness (1-5): How complete is the answer?
- Accuracy (1-5): How accurate is the information?

Provide scores and reasoning."""
    
    print("🔍 Scoring draft...")
    score = llm_with_structure.invoke([HumanMessage(content=prompt)])
    
    print(f"Scores - Clarity: {score.clarity}, Completeness: {score.completeness}, Accuracy: {score.accuracy}")
    print(f"Reasoning: {score.reasoning[:80]}...\n")
    
    # Build critique from low scores
    low_scores = []
    if score.clarity < QUALITY_THRESHOLD:
        low_scores.append(f"Clarity needs improvement (score: {score.clarity})")
    if score.completeness < QUALITY_THRESHOLD:
        low_scores.append(f"Completeness needs work (score: {score.completeness})")
    if score.accuracy < QUALITY_THRESHOLD:
        low_scores.append(f"Accuracy could be better (score: {score.accuracy})")
    
    critique = "; ".join(low_scores) if low_scores else "APPROVED"
    
    return {
        "scores": [score],
        "critique": critique,
        "iterations": state["iterations"] + 1
    }

def metric_finalizer(state: MetricReflectionState) -> dict:
    """Finalize with score summary."""
    print("\nReflection complete!\n")
    
    return {"final_output": state["draft"]}

In [ ]:
def should_metric_reflect(state: MetricReflectionState) -> Literal["generator", "finalizer"]:
    """Check if all scores meet threshold."""
    if not state.get("scores"):
        return "generator"
    
    latest_score = state["scores"][-1]
    all_good = (latest_score.clarity >= QUALITY_THRESHOLD and 
                latest_score.completeness >= QUALITY_THRESHOLD and 
                latest_score.accuracy >= QUALITY_THRESHOLD)
    
    if all_good or state["iterations"] >= MAX_METRIC_ITERATIONS:
        return "finalizer"
    
    return "generator"

In [ ]:
reflection_builder = StateGraph(MetricReflectionState)
reflection_builder.add_node("generator", metric_generator)
reflection_builder.add_node("critic", metric_critic)
reflection_builder.add_node("finalizer", metric_finalizer)

reflection_builder.add_edge(START, "generator")
reflection_builder.add_edge("generator", "critic")
reflection_builder.add_conditional_edges(
    "critic",
    should_metric_reflect,
    {"generator": "generator", "finalizer": "finalizer"}
)
reflection_builder.add_edge("finalizer", END)

reflection_agent = reflection_builder.compile()


In [ ]:

try:
    display(Image(reflection_agent.get_graph().draw_mermaid_png()))
except:
    print("Graph visualization requires mermaid support")

### Testing

In [ ]:
task1 = "Explain distributed power supply in simple terms"

result = reflection_agent.invoke({
    "task": task1,
    "draft": "",
    "iterations": 0,
    "scores": [],
    "critique": "",
    "final_output": ""
    
})

print(f"\n{'='*70}")
print("📊 FINAL OUTPUT (after reflection):")
print(f"{'='*70}")
print(result["final_output"])
print(f"\nTotal iterations: {result['iterations']}")
print(f"{'='*70}\n")

## Plan-Execute + Reflection Hybrid